Importing module

In [0]:
from __future__ import print_function

import csv
from os import listdir
import numpy as np
import tensorflow as tf

from math import tan, atan, sqrt, floor
from itertools import product
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from random import shuffle
from keras import Input, Model
from keras.callbacks import LearningRateScheduler
from keras.initializers import RandomNormal
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, CuDNNLSTM, Reshape
from keras.regularizers import l1, l2
from keras.optimizers import Adam

Using TensorFlow backend.


Mounting google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Loading data matrix (joints)

In [0]:
PATH_NPY = "/content/gdrive/My Drive/Gym_trainer/openpose/output/"
PATH_CSV = "/content/gdrive/My Drive/Gym_trainer/openpose/csv/"

npy_files = listdir(PATH_NPY)
npy_files.sort()
x_data = []
for f in npy_files:
  # x_data.append(np.load(PATH_NPY + f))
  path = PATH_CSV + f[0:3] + '.csv'
  #remove squat
  with open(path) as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
      if len(row) == 5:
        if row[0][0:6] == f[0:-4] and int(row[1]) == 8:
          indx = int(row[3])
          break
  temp = np.load(PATH_NPY + f) 
  x_data.append(temp[: , 0:indx, :])

Loading score vector

In [0]:
PATH_SCORE = "/content/gdrive/My Drive/Gym_trainer/openpose/"

y_data = []
with open(PATH_SCORE + "New.csv") as csv_file:
  csv_reader = csv.reader(csv_file)
  for row in csv_reader:
    y_data.append(float(row[1]))

In [0]:
def remove_bottom_joints(data):
  joints = {0,1,2,3,4,5,6,7,8,9,12,15,16,17,18}
  new_data = []
  #each video
  for i in range(len(data)):
    shape = data[i].shape
    shape = (shape[0],shape[1],15)
    x = np.zeros(shape)
    # each frame
    for j in range(shape[1]):
      #each joint
      l = 0
      for k in range(25):
        if k in joints:
          x[0,j,l] = data[i][0,j,k]
          l += 1
      #each joint
      l = 0
      for k in range(25):
        if k in joints:
          x[1,j,l] = data[i][1,j,k]
          l += 1

    new_data.append(x)
  return new_data

In [0]:
x_data = remove_bottom_joints(x_data)

Max_size function

In [0]:
def maximum_size(data, axis):
  max_size = 0
  for x in x_data:
    size = x.shape[axis]
    if size >= max_size:
      max_size = size
  return max_size

Padding function

In [0]:
def pad(data, max_size):
  data_pad = []
  first_shape, _, last_shape = data[0].shape
  for x in data:
    x_zeros = np.zeros((first_shape, max_size - x.shape[1], last_shape))
    new_x = np.append(x, x_zeros, axis = 1)
    data_pad.append(new_x)
  return data_pad

New pad

In [0]:
def new_pad(data, max_size):
  data_pad = []
  last_shape = data[0].shape[1]
  for x in data:
    x_zeros = np.zeros((max_size - x.shape[0], last_shape))
    new_x = np.append(x, x_zeros, axis = 0)
    data_pad.append(new_x)
  return data_pad

Standardize

In [0]:
#retrancher distance à 8 puis divisier par la norme

def standardize(data):
  new_data = data
  nb_joints = data[0].shape[2]
  for i in range(len(data)):
    for j in range(data[i].shape[1]):
      x_center = data[i][0,j,8]
      y_center = data[i][1,j,8]
      norm_sq = pow(data[i][0,j,1],2) + pow(data[i][1,j,1],2)
      for k in range(nb_joints):
        if k == 8:
          new_data[i][0,j,k] = 0
          new_data[i][1,j,k] = 0
        else:
          new_data[i][0,j,k] = pow(data[i][0,j,k] - x_center,2) / norm_sq
          new_data[i][1,j,k] = pow(data[i][1,j,k] - y_center,2) / norm_sq
  return new_data

New standardize

In [0]:
#retrancher distance à 8 puis divisier par la norme

def new_standardize(data):
  new_data = data
  nb_joints = data[0].shape[1]//2
  # each video
  for i in range(len(data)):
    # middle of hips
    x_center = data[i][0,8]
    y_center = data[i][1,23]
    # each frame
    for j in range(data[i].shape[0]):
      norm_sq = sqrt(pow(data[i][j,1],2) + pow(data[i][j,16],2))
      for k in range(nb_joints):
        # each joint
        if k == 8:
          new_data[i][j,8] = 0
          new_data[i][j,23] = 0
        else:
          if norm_sq != 0:
            new_data[i][j,k] = (data[i][j,k] - x_center) / norm_sq
            new_data[i][j,k + 15] = (data[i][j,k + 15] - y_center) / norm_sq
  return new_data

Changing shape

In [0]:
z = []
for x in x_data:
  temp = np.concatenate((x[0,:,:],x[1,:,:]), axis=1)
  z.append(temp)

x_data = z

Spliting data

In [0]:
def split_data(x_data, y_data, index):
  x_train = np.array(x_data[0:index], ndmin=2)
  x_test = np.array(x_data[(index + 1):])
  y_train = np.array(y_data[0:index])
  y_test = np.array(y_data[(index + 1):])
  return (x_train, x_test, y_train, y_test)

Sorting videos by size (number of frames)

In [0]:
def sort_by_size(x_data, y_data):
  shapes_index = [(i, x_data[i].shape[0]) for i in range(len(x_data))]
  dtype = [('index', int), ('shape', int)]
  shapes_index = np.array(shapes_index, dtype=dtype)
  shapes_index = np.sort(shapes_index, order='shape')
  x_data = [x_data[i] for i,shape in shapes_index]
  y_data = [y_data[i] for i,shape in shapes_index]
  return x_data

Shuffle

In [0]:
def shuffle_list(*ls):
  l = list(zip(*ls))
  shuffle(l)
  return l

def deshuffle(ls):
  x = []
  y = []
  for (i,j) in ls:
    x.append(i)
    y.append(j)
  return x, y

Processing data

In [0]:
x_data = sort_by_size(x_data, y_data)
x_data = x_data[-84:]
y_data = y_data[-84:]
max_size = maximum_size(x_data, 0)
x_data = new_pad(x_data, max_size)
x_data = new_standardize(x_data)
y_data = [y / 45 for y in y_data]
temp = shuffle_list(x_data, y_data)
x_data, y_data = deshuffle(temp)
index = int(len(x_data)*0.81)
x_train, x_test, y_train, y_test = split_data(x_data, y_data, index)

Create model

In [0]:
input_size = x_data[0].shape

def create_model(input_size, first_size, second_size):
  inputs = Input(shape=input_size, name='input')
  lstm_layer = CuDNNLSTM(first_size, kernel_initializer=RandomNormal(mean=0.0, stddev=0.01), kernel_regularizer=l2(0.1))(inputs)
  hidden_layer = Dense(second_size, activation='linear', kernel_initializer=RandomNormal(mean=0.0, stddev=0.01), kernel_regularizer=l2(0.1))(lstm_layer)
  outputs = Dense(1, activation='linear', kernel_initializer=RandomNormal(mean=0.0, stddev=0.01), kernel_regularizer=l2(0.1))(hidden_layer)
  model = Model(inputs, outputs)
  model.summary()
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
  return model

model = create_model(input_size, 1610, 16)

Learning rate decay

In [0]:
def step_decay(epoch):
  initial_lrate = 0.001
  drop = 0.5
  epochs_drop = 10.0
  lrate = initial_lrate * pow(drop, floor((1+epoch)/epochs_drop))
  return lrate

In [0]:
def print_history(history):
  """ summarize history for loss"""
  fiq, ax = plt.subplots(figsize=(8,8))
  ax.plot(history.history['loss'])
  ax.plot(history.history['val_loss'])
  ax.set(title='model loss', ylabel='loss', xlabel='epoch')
  ax.legend(['train', 'test'], loc='upper left')
  ax.grid(True, linestyle='-.')
  plt.show()

In [0]:
lrate = LearningRateScheduler(step_decay)
history = model.fit(x=x_train, y=y_train, batch_size=5,epochs=40, validation_data=(x_test, y_test), callbacks=[lrate], shuffle=False)
print_history(history)

In [0]:
def predict(x_test, y_test, model):
  """ print prediction on test set"""
  y_prediction = (np.array(model.predict(x_test))).flatten()
  print("Real value | Prediction")
  print("Correlation:")
  print(np.corrcoef(y_test, y_prediction))
  # print("Values")
  # for (y_predict, y) in zip(y_prediction, y_test):
  #   print(y, "|", y_predict)
  fiq, ax = plt.subplots(figsize=(8,8))
  ax.scatter(y_test, y_prediction)
  ax.set(title='Correlation', ylabel='predicted y_test', xlabel='real y_test')
  ax.grid(True, linestyle='-.')
  plt.show()

In [0]:
predict(x_test, y_test,model)

In [0]:
PATH_MODEL = "/content/gdrive/My Drive/Gym_trainer/openpose/model_config"

def for_model(input_size, low_bound, high_bound, low_bound2, high_bound2):
  # layer = list(map(lambda x: int(input_size[0]*input_size[1]//(10*x)), range(low_bound, high_bound)))
  layer = list(map(lambda x: int(pow(2,x)), range(low_bound, high_bound)))
  layer2 = list(map(lambda x: int(pow(2,x)), range(low_bound2, high_bound2)))
  corr_matrix = []
  for (i,j) in product(layer, layer2):
    model = create_model(input_size, i, j)
    lrate = LearningRateScheduler(step_decay)
    history = model.fit(x=x_train, y=y_train, epochs=80, validation_data=(x_test, y_test), callbacks=[lrate], shuffle=False, verbose=0)
    y_prediction = (np.array(model.predict(x_test))).flatten()
    correlation = np.corrcoef(y_test, y_prediction)
    correlation = correlation[0,1]
    print("Dimensions : ",i,"et",j,", correlation :", correlation)
    predict(x_test, y_test,model)
    corr_matrix.append((i,j,correlation))
    model.save(PATH_MODEL + "_" + str(i) + "_" + str(j) + '.h5')
  return corr_matrix


In [0]:
input_size = x_data[0].shape
matrix = for_model(input_size, 6,11, 4, 9)
print(matrix)